In [ ]:
import wandb
from QPNN import *
from medmnist import PneumoniaMNIST
dataset = PneumoniaMNIST(split="test", download=True)
import torchvision.transforms as transforms
import medmnist
import numpy as np
from medmnist import INFO, Evaluator
target_dataset="retinamnist"
dataset_list={"iris": 1,"digits": 2,"wine": 3,"cancer": 4, "iris_linear": 5, "moon": 6, "retinamnist": 7}
print(dataset_list)

In [ ]:
def main(gpu=False):
    x,xval,y,yval=get_dataset(dataset_list[target_dataset],split=True)
    net=QPNN([5],x,y,xval,yval)#,
    if not gpu:
        net.device="cpu"
    net.train(wandb_verbose=False,verbose=True)

In [ ]:
main()

In [ ]:
def generate_sweep_config():
    wandb.login()
    sweep_config = {'method': 'bayes'}
    metric = {'name': 'loss','goal': 'minimize'}
    sweep_config['metric'] = metric
    parameters_dict = { 'epochs': {
                            'values': [10,50,100,200]},
                       'optimizer':{'values':["SGD","ADAM","ADAMW","RMSPROP"]},
                        'arch_elements':{'values':[2,4,5,6,7,10]}
                        }
    sweep_config['parameters'] = parameters_dict
    parameters_dict.update({
        'learning_rate': {
            # a flat distribution between 0 and 0.1
            'distribution': 'uniform',
            'min': 0,
            'max': 0.9
          },
        'batch_size': {
            # integers between 32 and 256
            # with evenly-distributed logarithms 
            'distribution': 'q_log_uniform_values',
            'q': 2,
            'min': 8,
            'max': 100,
          }
        })
    pprint.pprint(sweep_config)
    return sweep_config

In [ ]:
sweep_id = wandb.sweep(sweep=generate_sweep_config(), entity='quantum_kets', project='QPNNTRAINSWEEP_retinaMNIST')
wandb.agent(sweep_id, function=main, count=200)